CLEAN SKILL

In [6]:
from __future__ import division
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import string
import time
import random
import os

# BASE_PATH = os.path.dirname(__file__)

class CleanSkills():

    def __init__(self, input_skill_file,  clean_skill_file, output_skill_file):
        if type(input_skill_file) is pandas.core.frame.DataFrame:
            self.input_skills_list = input_skill_file
        else:
            self.input_skills_list = pd.read_csv(input_skill_file)['skill']

        if type(clean_skill_file) is pandas.core.frame.DataFrame:
            self.clean_skills_list = clean_skill_file
        else:
            self.clean_skills_list = pd.read_csv(clean_skill_file)['skill']
        self.output_clean_skill_file = output_skill_file
        self.res = {}        

    def clean_skills(self, output_type='DataFrame'):
        result_df = pd.DataFrame()
        cols = ['skill', 'clean_skill']

        # self.input_skills_list = ['oracle database: 11g', 'self confidant', 'uv visible spectroscopy', 'junior software developer', 'angular material design', 'signal processsing']
        for skill in self.input_skills_list:
            skill = skill.strip()
            if skill.lower() in self.res.keys():
                skills_stat_df = pd.DataFrame({'skill':[skill], 'clean_skill':[self.res[skill.lower()]]})
                result_df = result_df.append(skills_stat_df)
                continue
            skill_list, clean_skill_list, total_score_list, ratio_list, partial_ratio_list,\
            sort_ratio_list, partial_sort_ratio_list, set_ratio_list, partial_set_ratio_list = [], [], [], [], [], [], [], [], []
            for clean_skill in self.clean_skills_list:
                clean_skill = clean_skill.lower()
                ratio = fuzz.ratio(skill, clean_skill)
                partial_ratio = fuzz.partial_ratio(skill, clean_skill)
                sort_ratio = fuzz.token_sort_ratio(skill, clean_skill)
                partial_sort_ratio = fuzz.partial_token_sort_ratio(skill, clean_skill)
                set_ratio = fuzz.token_set_ratio(skill, clean_skill)
                partial_set_ratio = fuzz.partial_token_set_ratio(skill, clean_skill)

                total_score = (1 * ratio) + (0.8 * set_ratio) + (0.8 * sort_ratio) + (0.5 * partial_ratio) + \
                              (0.3 * partial_set_ratio) + (0.3 * partial_sort_ratio)
                
                skill_list.append(skill)
                clean_skill_list.append(clean_skill)
                total_score_list.append(total_score)
                ratio_list.append(ratio)
                partial_ratio_list.append(partial_ratio)
                sort_ratio_list.append(sort_ratio)
                partial_sort_ratio_list.append(partial_sort_ratio)
                set_ratio_list.append(set_ratio)
                partial_set_ratio_list.append(partial_set_ratio)
            skills_stat_df = pd.DataFrame({'skill':skill_list, 'clean_skill':clean_skill_list,\
                            'total_score':total_score_list, 'ratio':ratio_list,\
                            'partial_ratio':partial_ratio_list, 'sort_ratio':sort_ratio_list,\
                            'partial_sort_ratio':partial_sort_ratio_list, 'set_ratio':set_ratio_list, 'partial_set_ratio':partial_set_ratio_list})\
                            .sort_values(by=['total_score', 'ratio', 'sort_ratio', 'set_ratio', 'partial_ratio'], ascending=False)[:1]
            self.res[skills_stat_df['skill'][0].lower()] = skills_stat_df['clean_skill'][0]
            result_df = result_df.append(skills_stat_df[cols])

        if output_type != 'DataFrame':
            result_df.to_csv(self.output_clean_skill_file)
        else:
            return result_df


LEMMATISE SKILLS

In [36]:
import os
import nltk
from nltk.stem import WordNetLemmatizer
import csv
import sys
import string

class LemmatiseSkills():
    
    def __init__(self):
        self.wordnet_lemmatizer = WordNetLemmatizer()
        # nltk.download('wordnet')

    def lemmatise_file(self, in_file, out_file):
        with open(in_file, 'r') as input_file, open(out_file, 'w') as output_file:
            line = input_file.readline()
            while line:
                flag = 0
                for char in line:
                    if ord(char) > 127:
                        flag = 1
                        break
                if flag:
                    output_file.write('" "\n')
                    line = input_file.readline()
                    continue
                line = line.translate(str.maketrans('-/"', '   ', '&()'))
                new_line = ''
                for word in line.split(' '):
                    word = word.strip().lower()
                    lemma = self.wordnet_lemmatizer.lemmatize(word)
                    new_line += word+' ' if len(word) <= 4 else lemma+' '
                line = input_file.readline()
                output_file.write('"{new_line}"\n'.format(new_line=new_line.strip()))


CLEANING DRIVER

In [50]:
verified_skills_file = '/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/verified_skills.csv'
not_verified_skills_file = '/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/input_unverified_skills.csv'
lemmatised_not_verified_skills_file = '/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/input_unverified_skills_lemmatised.csv'
verified_skills = pd.read_csv(verified_skills_file)
vs1 = verified_skills[verified_skills['Status']=='Verified']['Name']
vs2 = verified_skills[verified_skills['New Status']=='Verified']['New Name']
verified_skills = pd.concat([vs1, vs2])
verified_skills = pd.DataFrame(verified_skills)

not_verified_skills = pd.read_csv(verified_skills_file)
not_verified_skills = not_verified_skills[(not_verified_skills['Status'] != 'Verified')\
                                          & (not_verified_skills['New Status'] != 'Verified')]
not_verified_skills['Name'].to_csv(not_verified_skills_file, index=False)

ls = LemmatiseSkills()
ls.lemmatise_file(not_verified_skills_file, lemmatised_not_verified_skills_file)
lemmatised_not_verified_skills = pd.read_csv(lemmatised_not_verified_skills_file, header=None)

# print(not_verified_skills.head(3), lemmatised_not_verified_skills.head(3))
not_verified_skills['lemmatised_skill'] = lemmatised_not_verified_skills
display(not_verified_skills)
not_verified_skills.to_csv('/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/temp.csv')

/home/jasmeet16-jtg/projects/virtualenvs/calyx_ds3/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  del sys.path[0]


,ID,Name,Similar To,Status,New Status,New Name,New Similar To,lemmatised_skill
0,25732.0,`,NaN,Not Verified,NaN,NaN,NaN,`
1,42615.0,=,NaN,Not Verified,NaN,NaN,NaN,=
2,13613.0,-,NaN,Not Verified,NaN,NaN,NaN,NaN
3,7949.0,-$-$-$-$#;#;#;++,NaN,Not Verified,NaN,NaN,NaN,$ $ $ $#;#;#;++
4,5884.0,/,NaN,Not Verified,NaN,NaN,NaN,NaN
5,27581.0,.,NaN,Not Verified,NaN,NaN,NaN,.
6,47309.0,.....,NaN,Not Verified,NaN,NaN,NaN,.....
7,59495.0,..........,NaN,Not Verified,NaN,NaN,NaN,..........
8,59496.0,...........,NaN,Not Verified,NaN,NaN,NaN,...........
9,59500.0,..................................,NaN,Not Verified,NaN,NaN,NaN,..................................


In [42]:
lemmatised_not_verified_skills.columns

Index(['`'], dtype='object')